In [74]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras.preprocessing import sequence
import seaborn as sns
import matplotlib.pyplot as plt

In [75]:
# reads all .txt files in the path given and adds them to the list
def read_files(relative_path, ls):
    files = os.listdir(relative_path)
    files = list(filter(lambda x: x[-4:] == '.txt', files))

    for file in files:
        name = relative_path + "/" + file
        with open(name, 'r') as f:
            ls.append(f.read().replace('\n', ''))


In [95]:
#reading all data from different data sets
fake_data1 = pd.read_csv("../data/Fake and real news dataset/Fake.csv")
true_data1 = pd.read_csv("../data/Fake and real news dataset/True.csv")

#from another source - obtaining the contents of each .txt file as a string
fake_data2 = []
true_data2 = []

read_files("../data/fakeNewsDatasets/fakeNewsDataset/fake", fake_data2)     
read_files("../data/fakeNewsDatasets/celebrityDataset/fake", fake_data2)
read_files("../data/fakeNewsDatasets/fakeNewsDataset/legit", true_data2)
read_files("../data/fakeNewsDatasets/celebrityDataset/legit", true_data2)




In [100]:
#fake_data1.head()
#true_data1.head()
print(true_data1.isnull().values.any())
print(fake_data1.isnull().values.any())

False
False


In [103]:
#creating and merging dataframes/lists into 1 data frame
is_fake = [1 for x in range(len(fake_data1))]
fake_data1 = pd.DataFrame(list(fake_data1['text']), columns=['text'])
fake_data1['is_fake'] = is_fake
#print(fake_data1.head())

is_fake = [0 for x in range(len(true_data1))]
true_data1 = pd.DataFrame(list(true_data1['text']), columns=['text'])
true_data1['is_fake'] = is_fake
#print(true_data1.head())


is_fake = [1 for x in range(len(fake_data2))]
fake_data2 = pd.DataFrame(fake_data2, columns=['text'])
fake_data2['is_fake'] = is_fake

is_fake = [0 for x in range(len(true_data2))]
true_data2 = pd.DataFrame(true_data2, columns=['text'])
true_data2['is_fake'] = is_fake


#joining all data sets
frames = [true_data1, fake_data1, fake_data2, true_data2]
merged = pd.concat(frames)

merged.describe
merged = sk.utils.shuffle(merged)
print(merged.isnull().values.any())
#df.head()

False


In [105]:
#splitting data into training and tests sets
split = 0.2
train_txts, test_txts, train_res, test_res = train_test_split(merged['text'], merged['is_fake'], test_size=split, random_state=12)